In [ ]:
from THB.space import Space
from THB.THB_core import *
from THB.datastructures import *
from THB.funcs import *
from THB.THB_utils import *
from THB.THB_torch import *
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
bs1 = BSpline(knotvector=np.array([0, 0, 0, 0.2, 0.4, 0.6, 0.8, 1, 1, 1]), degree=2)
# bs2 = BSpline(knotvector=np.array([0, 0, 0, 0.2, 0.4, 0.6, 0.8, 1, 1, 1]), degree=2)

bs2 = BSpline(knotvector=np.array([0, 0, 0, 0, 0.2, 0.4, 0.6, 0.8, 1, 1, 1, 1]), degree=3)
bs3 = BSpline(knotvector=np.array([0, 0, 0, 0.33, 0.66, 0.8, 1, 1, 1]), degree=2)

tp = TensorProduct(bsplines=[bs1, bs2, bs3])

In [ ]:
h_space = Space(tensor_product=tp, num_levels=3)
h_space.build_hierarchy_from_domain_sequence()
ac_cells = compute_active_cells_active_supp(h_space.cells, h_space.fns, h_space.degrees)
fn_coeffs = compute_fn_projection_matrices(h_space.fns, h_space.Coeff, h_space.degrees)

In [ ]:
coo = generate_parametric_coordinates((100,100,100))
CP0 = grevilleAbscissae(h_space.sh_fns[0], h_space.degrees, h_space.knotvectors[0])

ndim = h_space.ndim
ctrl_pts = {lev: np.zeros((*h_space.sh_fns[lev], ndim)) for lev in range(3)}
ctrl_pts[0] = CP0

CP = ControlPoints(h_space)
ac_spans = compute_active_span(coo, h_space.knotvectors, h_space.cells, h_space.degrees, h_space.sh_fns)
PHI, num_supp = compute_basis_fns_tp(coo, ac_spans, ac_cells, fn_coeffs, h_space.sh_fns, h_space.knotvectors, h_space.degrees)

# eval_data = prepare_data_for_acceleration(PHI, ac_spans, num_supp, ctrl_pts, ac_cells, h_space.sh_fns, device='cuda')

# THBEval.apply(*eval_data)

In [ ]:
h_space.refine_basis_fn((3, 3, 0), 0)
h_space.build_hierarchy_from_domain_sequence()
# h_space.refine_basis_fn((4,4), 1)
# h_space.build_hierarchy_from_domain_sequence()

ac_cells = compute_active_cells_active_supp(h_space.cells, h_space.fns, h_space.degrees)
fn_coeffs = compute_fn_projection_matrices(h_space.fns, h_space.Coeff, h_space.degrees)

# ac_spans = compute_active_span(coo, h_space.knotvectors, h_space.cells, h_space.degrees, h_space.sh_fns)
# PHI, num_supp = compute_tensor_product(coo, ac_spans, ac_cells, fn_coeffs, h_space.sh_fns, h_space.knotvectors, h_space.degrees)
# print(ac_spans[5555], PHI[5555], ac_cells[1][(5, 5)])

# plot1 = THB_plot(dir='./', figname='gridplot')
# plot1.add_2Daxis(axis_name=0)
# plot2DGrid(plot1.ax[0], h_space.cells, h_space.knotvectors, h_space.degrees)
# plot1.save_fig()

# plot3DGrid(h_space.cells, h_space.knotvectors)

In [ ]:
ctrl_pts = CP.update_CP(ctrl_pts, h_space.fns, h_space.Coeff)
plot3DAdaptiveGrid(ac_cells, ctrl_pts, h_space.knotvectors, fn_coeffs, h_space.sh_fns, h_space.degrees)
# output = evaluate(PHI, ctrl_pts, ac_spans, num_supp, ac_cells)
# plt.scatter(output[:,0], output[:,1])
# cp0 = ctrl_pts[0].reshape(-1, 2)
# cp1 = ctrl_pts[1].reshape(-1, 2)
# CP0 = CP0.reshape(-1, 2)
# plt.scatter(CP0[:, 0], CP0[:, 1])
# plt.scatter(cp1[:,0], cp1[:,1])